In [1]:
# -*- coding: utf-8 -*-
# 2020/08/01 バグの修正
import re

#他のtexファイルにコピペするときにラベル名を変えて競合するのを防ぐ
bridge = ''

#処理済みの式番号が後続の処理で検知され再び処理されるのを防ぐ
virgin = 'Virgin'

# ファイルをオープンする
f = open("report.tex", "r", encoding="utf-8")

# すべての内容を読み込む
contents = f.read()

documentclass = 'article'
if contents.find('book', 0, 100) != -1:
	documentclass = 'book'

# ファイルをクローズする
f.close()

chpCount = 0
secCount = 0
subCount = 0
sssCount = 0
eqnCount = 0
figCount = 0
tabCount = 0

end = 0
appendixflag = False

start0 = 1000000
totalchara = len(contents)
if start0 < totalchara:
	print('start0={}が総字数{}を上回るようにしてください'.format(start0, totalchara))

appendNum = ['', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
subeqnNum = ['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n']

while True:
	start=start0
	flag = ''

	#処理対象の開始位置を決定
	if -1 < contents.find('\\chapter{', end):
		start = contents.find('\\chapter{', end)+len('\\chapter{')+1
		flag = 'chp'
	if -1 < contents.find('\\section{', end) < start :
		start = contents.find('\\section{', end)+len('\\section{')+1
		flag = 'sec'
	if -1 < contents.find('\\subsection{', end) < start:
		start = contents.find('\\subsection{', end)+len('\\subsection{')+1
		flag = 'sub'
	if -1 < contents.find('\\subsubsection{', end) < start:
		start = contents.find('\\subsubsection{', end)+len('\\subsubsection{')+1
		flag = 'sss'
	if -1 < contents.find('\\begin{equation}', end) < start:
		start = contents.find('\\begin{equation}', end)+len('\\begin{equation}')+1
		endn = contents.find('\\end{equation}', start)
		flag = 'eqn'
	if -1 < contents.find('\\begin{eqnarray}', end) < start:
		start = contents.find('\\begin{eqnarray}', end)+len('\\begin{eqnarray}')+1
		endn = contents.find('\\end{eqnarray}', start)
		flag = 'eqn'
	if -1 < contents.find('\\begin{align}', end) < start:
		start = contents.find('\\begin{align}', end)+len('\\begin{align}')+1
		endn = contents.find('\\end{align}', start)
		flag = 'eqn'
	if -1 < contents.find('\\begin{figure}', end) < start:
		start = contents.find('\\begin{figure}', end)+len('\\begin{figure}')+1
		endn = contents.find('\\end{figure}', start)
		flag = 'fig'
	if -1 < contents.find('\\begin{figure*}', end) < start:
		start = contents.find('\\begin{figure*}', end)+len('\\begin{figure*}')+1
		endn = contents.find('\\end{figure*}', start)
		flag = 'fia'
	if -1 < contents.find('\\begin{table}', end) < start:
		start = contents.find('\\begin{table}', end)+len('\\begin{table}')+1
		endn = contents.find('\\end{table}', start)
		flag = 'tab'

	subequMode=False
	if -1 < contents.find('\\begin{subequations}', end) < start:
		subequMode=True

	#appendix
	if -1 < contents.find('\\appendix', end) < start:
		end = contents.find('\\appendix', end)+len('\\appendix')+1

		#記録をターミナルに表示
		if documentclass == 'book':
				print('第{0}章 式{1} 図{2} 表{3}'.format(chpCount, eqnCount, figCount, tabCount))
		elif documentclass == 'article':
				print('第{0}節 式{1} 図{2} 表{3}'.format(secCount, eqnCount, figCount, tabCount))

		if documentclass == 'book':
			appendixflag = True
			chpCount = 0
			secCount = 0
			subCount = 0
			eqnCount = 0
			sssCount = 0
			figCount = 0
			tabCount = 0
		elif documentclass == 'article':
			appendixflag = True
			secCount = 0
			subCount = 0
			sssCount = 0

	#新しいラベルを発見できない場合は終了
	if start0 <= start:
		break

	#処理対象の終了位置を決める
	if flag in {'chp', 'sec', 'sub', 'sss'}:
		end = contents.find('\n', start)
	if flag in {'eqn', 'fig', 'fia', 'tab'}:
		end = endn

	#ラベルを作成
	if flag == 'chp':

		#記録をターミナルに表示
		if documentclass == 'book' and 1 <= chpCount:
			if appendixflag == False:
				print('第{0}章 式{1} 図{2} 表{3}'.format(chpCount, eqnCount, figCount, tabCount))
			else:
				print('第{0}章 式{1} 図{2} 表{3}'.format(appendNum[chpCount], eqnCount, figCount, tabCount))

		chpCount += 1
		secCount = 0
		subCount = 0
		sssCount = 0
		eqnCount = 0
		figCount = 0
		tabCount = 0
		if appendixflag == False:
			label = 'chp'+bridge+virgin+str(chpCount)
		else:
			label = 'chp'+bridge+virgin+appendNum[chpCount]
			
	elif flag == 'sec':

		#記録をターミナルに表示
		if documentclass == 'article' and 1 <= secCount:
			if appendixflag == False:
				print('第{0}節 式{1} 図{2} 表{3}'.format(secCount, eqnCount, figCount, tabCount))
			else:
				print('第{0}節 式{1} 図{2} 表{3}'.format(appendNum[secCount], eqnCount, figCount, tabCount))

		secCount += 1
		subCount = 0
		sssCount = 0
		if documentclass == 'book':
			if appendixflag == False:
				label = 'sec'+bridge+virgin+str(chpCount)+'.'+str(secCount)
			else:
				label = 'sec'+bridge+virgin+appendNum[chpCount]+'.'+str(secCount)
		if documentclass == 'article':
			if appendixflag == False:
				label = 'sec'+bridge+virgin+str(secCount)
			else:
				label = 'sec'+bridge+virgin+appendNum[secCount]

	elif flag == 'sub':
		subCount += 1
		sssCount = 0
		if documentclass == 'book':
			if appendixflag == False:
				label = 'sub'+bridge+virgin+str(chpCount)+'.'+str(secCount)+'.'+str(subCount)
			else:
				label = 'sub'+bridge+virgin+appendNum[chpCount]+'.'+str(secCount)+'.'+str(subCount)
		if documentclass == 'article':
			if appendixflag == False:
				label = 'sub'+bridge+virgin+str(secCount)+'.'+str(subCount)
			else:
				label = 'sub'+bridge+virgin+appendNum[secCount]+'.'+str(subCount)

	elif flag == 'sss':
		sssCount += 1
		if documentclass == 'article':
			if appendixflag == False:
				label = 'sub'+bridge+virgin+str(secCount)+'.'+str(subCount)+'.'+str(sssCount)
			else:
				label = 'sub'+bridge+virgin+appendNum[secCount]+'.'+str(subCount)+'.'+str(sssCount)

	elif flag == 'fig':
		figCount += 1
		if documentclass == 'book':
			if appendixflag == False:
				label = 'fig'+bridge+virgin+str(chpCount)+'.'+str(figCount)
			else:
				label = 'fig'+bridge+virgin+appendNum[chpCount]+'.'+str(figCount)
		if documentclass == 'article':
			label = 'fig'+bridge+virgin+str(figCount)
    
	elif flag == 'fia':
		figCount += 1
		if documentclass == 'book':
			if appendixflag == False:
				label = 'fig'+bridge+virgin+str(chpCount)+'.'+str(figCount)
			else:
				label = 'fig'+bridge+virgin+appendNum[chpCount]+'.'+str(figCount)
		if documentclass == 'article':
			label = 'fig'+bridge+virgin+str(figCount)

	elif flag == 'tab':
		tabCount += 1
		if documentclass == 'book':
			if appendixflag == False:
				label = 'tab'+bridge+virgin+str(chpCount)+'.'+str(tabCount)
			else:
				label = 'tab'+bridge+virgin+appendNum[chpCount]+'.'+str(tabCount)
		if documentclass == 'article':
			label = 'tab'+bridge+virgin+str(tabCount)

	#ラベルを処理
	if (documentclass == 'book' and  flag in {'chp', 'sec', 'sub'}) or (documentclass == 'article' and  flag in {'sec', 'sub', 'sss'} ):
		#ラベルがない場合はラベルを新設
		if contents.find('\\label{', start, end) == -1:
			contents = contents[0:end]+' \label{'+label+'}'+contents[end:len(contents)]
		else:
		#ラベルがある場合はラベル番号を更新
			split1 = contents[0:start]
			split2 = contents[start:end]
			split3 = contents[end:len(contents)]

			#refのラベルを変更する
			s0=re.search('label{.+}', split2).group(0)
			s1=s0.split('{')
			s2=s1[1].split('}')
			split1=split1.replace('\\ref{'+s2[0]+'}', '\\ref{'+label+'}')
			split3=split3.replace('\\ref{'+s2[0]+'}', '\\ref{'+label+'}')

			split2 = re.sub('label{.+}', 'label{'+label+'}', split2)
			contents = split1+split2+split3
	
	#bookなのにsssにラベルがある場合はラベル番号を除去
	if documentclass == 'book' and flag == 'sss':
		if contents.find('\\label{', start, end) != -1:
			split1 = contents[0:start]
			split2 = contents[start:end]
			split3 = contents[end:len(contents)]
			split2 = re.sub(' \\\\label{.+}', '', split2)
			split2 = re.sub('\\\\label{.+}', '', split2)
			contents = split1+split2+split3

	if flag == 'eqn':
		firstFlag = True
		split1 = contents[0:start]
		split2 = contents[start:end]
		split3 = contents[end:len(contents)]
		split2n = ''
		subeqnCount = 0

		if split2.find('Qcircuit') == -1:
			split2 = split2.split('\\\\')
			matrixMode = 0
			subequ = 0
			for split2e in split2:
				#matrixModeは行列外ならば偶数, 行列内ならば奇数
				matrixMode += split2e.count('array')
				
				if matrixMode % 2 == 0:
					if split2e.find('nonumber') != -1:
						#nonumberがある場合
						if split2[-1] != split2e:
							#最後の要素でない場合
							split2e = split2e + '\\\\'
					else:
						#nonumberがない場合
						#ラベルを生成
						if documentclass == 'book':
							if subequMode == False:
								eqnCount += 1
								if appendixflag == False:
									label = 'eqn'+bridge+virgin+str(chpCount)+'.'+str(eqnCount)
								else:
									label = 'eqn'+bridge+virgin+appendNum[chpCount]+'.'+str(eqnCount)
							else:
								subeqnCount += 1
								if firstFlag == True:
									eqnCount += 1
									firstFlag = False
								if appendixflag == False:
									label = 'eqn'+bridge+virgin+str(chpCount)+'.'+str(eqnCount)+subeqnNum[subeqnCount]
								else:
									label = 'eqn'+bridge+virgin+appendNum[chpCount]+'.'+str(eqnCount)+subeqnNum[subeqnCount]
						if documentclass == 'article':
							if subequMode == False:
								eqnCount += 1
								label = 'eqn'+bridge+virgin+str(eqnCount)
							else:
								subeqnCount += 1
								if firstFlag == True:
									eqnCount += 1
									firstFlag = False
								label = 'eqn'+bridge+virgin+str(eqnCount)+subeqnNum[subeqnCount]

						if split2e.find('\\label{') != -1:
							#ラベルがすでに存在する場合は式番号を更新
							#refのラベルを変更する
							s0=re.search('label{.+}', split2e).group(0)
							s1=s0.split('{')
							s2=s1[1].split('}')
							split1=split1.replace('\\ref{'+s2[0]+'}', '\\ref{'+label+'}')
							split3=split3.replace('\\ref{'+s2[0]+'}', '\\ref{'+label+'}')
							if split2[-1] != split2e:
								#最後の要素でない場合
								split2e = re.sub('label{.+}', 'label{'+label+'}', split2e)
								split2e = split2e + '\\\\'
							else:
								split2e = re.sub('label{.+}', 'label{'+label+'}', split2e)
						else:
							#ラベルがついていない場合はラベルを新設
							if split2[-1] != split2e:
								#最後の要素でないとき
								split2e = split2e + '\\label{'+label+'}\\\\'
							else:
								split2e = re.sub('\n$', '\\\\label{'+label+'}\n', split2e)
				else:# matrixMode
					split2e = split2e + '\\\\'
				split2n = split2n+split2e
			contents = split1+split2n+split3
		else:#Qcircuit
			contents = split1+split2+split3

	if flag in {'fig', 'fia', 'tab'}:
		split1 = contents[0:start]
		split2 = contents[start:end]
		split3 = contents[end:len(contents)]

		#refのラベルを変更する
		s0=re.search('label{.+}', split2).group(0)
		s1=s0.split('{')
		s2=s1[1].split('}')
		split1=split1.replace('\\ref{'+s2[0]+'}', '\\ref{'+label+'}')
		split3=split3.replace('\\ref{'+s2[0]+'}', '\\ref{'+label+'}')

		split2 = re.sub('label{.+}', 'label{'+label+'}', split2)
		contents = split1+split2+split3

#結果を表示
if documentclass == 'book':
	if appendixflag == False:
		print('第{0}章 式{1} 図{2} 表{3}'.format(chpCount, eqnCount, figCount, tabCount))
	else:
		print('第{0}章 式{1} 図{2} 表{3}'.format(appendNum[chpCount], eqnCount, figCount, tabCount))

if documentclass == 'article':
	if appendixflag == False:
		print('第{0}節 式{1} 図{2} 表{3}'.format(secCount, eqnCount, figCount, tabCount))
	else:
		print('第{0}節 式{1} 図{2} 表{3}'.format(appendNum[secCount], eqnCount, figCount, tabCount))

# Virginを削除
contents=contents.replace('Virgin', '')

# ファイルをオープンする
f = open("report.tex", "w", encoding="utf-8")

# すべての内容を書き込む
f.write(contents)

# ファイルをクローズする
f.close()


第1節 式0 図0 表0
第2節 式4 図3 表0
第3節 式4 図3 表0
第4節 式4 図3 表0
第5節 式4 図7 表0
第6節 式4 図7 表0
第7節 式4 図7 表0
第8節 式4 図7 表0
第9節 式4 図7 表0
第10節 式5 図7 表0
第11節 式5 図7 表0
第12節 式5 図7 表0
第13節 式5 図7 表0
